# Final NLP - TDTU
- Train GPU P100

# Environment

Install environment and declare global variable

In [ ]:
# base model from huggingFace or path to model
base_model = "minhtt/vistral-7b-chat" # Clone Clone of Viet-Mistral/Vistral-7B-Chat - request.

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
huggingface_write_token = user_secrets.get_secret("vuSnape")
wandb_key = user_secrets.get_secret("wandb-key")

In [ ]:
!pip install -U -q accelerate peft transformers trl bitsandbytes datasets

In [ ]:
!pip install -q datasets evaluate rouge_score

In [ ]:
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os
import torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import wandb
import pprint
import re
import datetime
from sklearn.model_selection import train_test_split

from evaluate import load
from sklearn.metrics import f1_score

# Configurations for Pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Login to HuggingFace

HuggingFace is a place where we store our base model and finetuned model adapter. It also provide us libraries to work with transformer model.

In [ ]:
from huggingface_hub import login
login(huggingface_write_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Login to wandb

Wandb, short for Weights & Biases, is a platform specifically designed for machine learning practitioners. It provides tools to streamline the experiment tracking and visualization process  for machine learning projects.

Instead of using tensorboard to track the trainning process, we use wandb to track and share online.

In [ ]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [ ]:
# Thiết lập biến môi trường cho WandB
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # Hoặc 'end'

# Đăng nhập vào WandB  # Thay thế bằng key thực của bạn
wandb.login(key=wandb_key)

# Khởi tạo một run mới với tên dự án có dấu thời gian hiện tại
now = datetime.datetime.utcnow()
run = wandb.init(
    project=f"Fine tuning vistral 7B with banking data {str(now).replace(':', '.')}",
    job_type="training",
    anonymous="allow"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hoangvu01639337130 (hoangvu01639337130-ton-duc-thang-university). Use `wandb login --relogin` to force relogin


# Functions
- Parameters

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# print(print_number_of_trainable_model_parameters(model))

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Tính toán ROUGE
    rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Tính toán F1 (đối với câu trả lời dạng chuỗi)
    f1 = f1_score([label for label in decoded_labels], [pred for pred in decoded_preds], average='weighted')

    return {
        "rouge1": rouge_results["rouge1"].mid.fmeasure,
        "rouge2": rouge_results["rouge2"].mid.fmeasure,
        "rougeL": rouge_results["rougeL"].mid.fmeasure,
        "f1": f1
    }

# Build Dataset

The dataset is crawl from website of VietinBank. These are public data. Thanks Nguyen Pham Tu Anh for preparing these data. If you have others data in Vietnamese related to banking domain, please share with us.

In [ ]:
# Origin data
dataset_org = load_dataset("HoangVuSnape/Vietnamese_Banking_QA")
df = pd.DataFrame(dataset_org['train'])
df

Generating train split:   0%|          | 0/814 [00:00<?, ? examples/s]

question  \
0           Tên đăng ký tiếng Việt của VietinBank là gì?   
1            Tên đăng ký tiếng Anh của VietinBank là gì?   
2                    Tên giao dịch của VietinBank là gì?   
3       Số giấy phép thành lập của VietinBank là số mấy?   
4           Mã số doanh nghiệp của VietinBank là số mấy?   
..                                                   ...   
809  Địa chỉ email liên hệ của ngân hàng Vietbank là gì?   
810        Ngân hàng BIDV có hỗ trợ qua phương pháp nào?   
811      Địa chỉ website của ngân hàng VietinBank là gì?   
812    Số điện thoại liên hệ của ngân hàng HDBank là gì?   
813          Ngân hàng VIB có hỗ trợ qua LinkedIn không?   

                                                                                                 answer  
0                                                     NGÂN HÀNG THƯƠNG MẠI CỔ PHẦN CÔNG THƯƠNG VIỆT NAM  
1                                            VIETNAM JOINT STOCK COMMERCIAL BANK FOR INDUSTRY AND TRADE  
2                                                                                            VietinBank  
3                                                Số 13/GP-NHNN do Ngân hàng Nhà nước cấp ngày 17/6/2022  
4                                                                                            0100111948  
..                                                                                                  ...  
809     Địa chỉ email liên hệ của Vietbank là contact@vietbank.vn, dùng để yêu cầu hỗ trợ và thông tin.  
810                                   BIDV hỗ trợ qua số điện thoại, email, và trang web của ngân hàng.  
811  Website chính thức của VietinBank là www.vietinbank.vn, cung cấp thông tin về dịch vụ và sản phẩm.  
812         Số điện thoại liên hệ của HDBank là 1900 6060, dùng để giải đáp thắc mắc và yêu cầu hỗ trợ.  
813               VIB không hỗ trợ qua LinkedIn, nhưng bạn có thể liên hệ qua số điện thoại hoặc email.  

[814 rows x 2 columns]

In [ ]:
# Transform to input pattern of model with some special token
df['text'] = '<s> [INST] ' + df['question'] +' [/INST]'+ df['answer'] + '</s>'

# Split dataset into train and validation sets
train_df, val_df = train_test_split(df[['text']], test_size=0.1, random_state=42)

# Convert to Huggingface Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
df.head(5)

question  \
0      Tên đăng ký tiếng Việt của VietinBank là gì?   
1       Tên đăng ký tiếng Anh của VietinBank là gì?   
2               Tên giao dịch của VietinBank là gì?   
3  Số giấy phép thành lập của VietinBank là số mấy?   
4      Mã số doanh nghiệp của VietinBank là số mấy?   

                                                       answer  \
0           NGÂN HÀNG THƯƠNG MẠI CỔ PHẦN CÔNG THƯƠNG VIỆT NAM   
1  VIETNAM JOINT STOCK COMMERCIAL BANK FOR INDUSTRY AND TRADE   
2                                                  VietinBank   
3      Số 13/GP-NHNN do Ngân hàng Nhà nước cấp ngày 17/6/2022   
4                                                  0100111948   

                                                                                                                            text  
0           <s> [INST] Tên đăng ký tiếng Việt của VietinBank là gì? [/INST]NGÂN HÀNG THƯƠNG MẠI CỔ PHẦN CÔNG THƯƠNG VIỆT NAM</s>  
1   <s> [INST] Tên đăng ký tiếng Anh của VietinBank là gì? [/INST]VIETNAM JOINT STOCK COMMERCIAL BANK FOR INDUSTRY AND TRADE</s>  
2                                                           <s> [INST] Tên giao dịch của VietinBank là gì? [/INST]VietinBank</s>  
3  <s> [INST] Số giấy phép thành lập của VietinBank là số mấy? [/INST]Số 13/GP-NHNN do Ngân hàng Nhà nước cấp ngày 17/6/2022</s>  
4                                                  <s> [INST] Mã số doanh nghiệp của VietinBank là số mấy? [/INST]0100111948</s>

In [ ]:
train_dataset['text'][:5]

['<s> [INST] Ngân hàng Eximbank có hỗ trợ qua Zalo không? [/INST]Eximbank không hỗ trợ qua Zalo, bạn có thể liên hệ qua số điện thoại hoặc email của ngân hàng.</s>',
 "<s> [INST] Thương hiệu mới 'VietinBank' được ra mắt vào năm nào? [/INST]Thương hiệu mới 'VietinBank' được ra mắt vào năm 2008.</s>",
 '<s> [INST] Công ty TNHH MTV Quản lý quỹ VietinBank Capital được thành lập vào năm nào? [/INST]Công ty TNHH MTV Quản lý quỹ VietinBank Capital được thành lập vào năm 2010.</s>',
 '<s> [INST] Ngân hàng nào có dịch vụ tư vấn tài chính miễn phí? [/INST]Vietcombank cung cấp dịch vụ tư vấn tài chính miễn phí cho khách hàng.</s>',
 '<s> [INST] Số tài khoản tối thiểu để mở tài khoản thanh toán tại VietinBank là bao nhiêu? [/INST]Số tài khoản tối thiểu để mở tài khoản thanh toán tại VietinBank là 50.000 VND, giúp khách hàng dễ dàng bắt đầu sử dụng dịch vụ.</s>']

In [ ]:
val_dataset['text'][:5]

['<s> [INST] VietinBank iPay có tính năng nhắc nợ thanh toán hóa đơn không? [/INST]Có, VietinBank iPay hỗ trợ dịch vụ nhắc nợ thanh toán hóa đơn, giúp bạn không bỏ lỡ các kỳ thanh toán quan trọng.</s>',
 '<s> [INST] Vietcombank có chương trình học bổng cho học sinh không? [/INST]Vietcombank tổ chức các chương trình học bổng cho học sinh xuất sắc, hỗ trợ tài chính và khuyến khích học tập.</s>',
 '<s> [INST] Cách thanh toán hóa đơn mua sắm và các dịch vụ tiện ích bằng VietinBank iPay như thế nào? [/INST]Bạn có thể thanh toán hóa đơn mua sắm và các dịch vụ tiện ích bằng VietinBank iPay thông qua ứng dụng trên điện thoại hoặc máy tính bảng, không cần sử dụng tiền mặt hay thẻ ATM.</s>',
 '<s> [INST] Có bao nhiêu chuyên gia phân tích tham gia Hội nghị? [/INST]Gần 80 chuyên gia phân tích đến từ hơn 60 tổ chức tham gia Hội nghị.</s>',
 '<s> [INST] VietinBank có hỗ trợ khách hàng mở tài khoản doanh nghiệp không? [/INST]VietinBank hỗ trợ khách hàng mở tài khoản doanh nghiệp với các dịch vụ tài c

# Load the model

In [ ]:

# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False  # Silence the warnings. Please re-enable for inference!
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

# Test model before finetune
- zero shot
- Few shot

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
def ask_chatbot(prompt):
    prompt = build_prompt(prompt)
    result = pipe(prompt)
    return result

def build_prompt(question):
  prompt=f"<s>[INST] {question} [/INST]"
  return prompt

In [ ]:
print(ask_chatbot("Hãy giới thiệu về VietinBank"))
print("---")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '<s>[INST] Hãy giới thiệu về VietinBank [/INST] Q:\n\nBạn có thể giải thích sự khác biệt giữa các loại hình bảo hiểm khác nhau không?\n\nA:\n\nBảo hiểm là một công cụ tài chính giúp bảo vệ cá nhân hoặc tổ chức khỏi những tổn thất tài chính tiềm tàng. Có nhiều loại bảo hiểm khác nhau, mỗi loại có đặc điểm và mục đích riêng. Dưới đây là một số loại bảo hiểm phổ biến nhất:\n\n1. Bảo hiểm nhân thọ: Loại bảo hiểm này cung cấp sự bảo vệ tài chính cho người thụ hưởng trong trường hợp người được bảo hiểm qua đời. Bảo hiểm nhân thọ có thể là vĩnh viễn hoặc có thời hạn và có thể được sử dụng để trả các khoản nợ, chi phí tang lễ hoặc các chi phí khác.\n\n2. Bảo hiểm y tế: Loại bảo hiểm này giúp trang trải các chi phí y tế, chẳng hạn như chi phí nằm viện, phẫu thuật hoặc'}]
---


In [ ]:
print(ask_chatbot("Hãy nói về ipay"))
print("---")

[{'generated_text': '<s>[INST] Hãy nói về ipay [/INST] Q:\n\nLàm thế nào để tôi có thể tạo một hàm để tính tổng của một danh sách các số?\n\nTôi muốn tạo một hàm có tên là "sum_list" nhận một danh sách các số làm đầu vào và trả về tổng của các số đó.\n\nDưới đây là một ví dụ về cách sử dụng hàm:\n\n```\nsum_list([1, 2, 3, 4, 5])\n```\n\nHàm này sẽ trả về tổng của các số trong danh sách, trong trường hợp này là 15.\n\nDưới đây là một ví dụ khác:\n\n```\nsum_list([10, 20, 30, 40, 50])\n```\n\nHàm này sẽ trả về tổng của các số trong danh sách, trong trường hợp này là 1'}]
---


In [ ]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 314585088
all model parameters: 3804246016
percentage of trainable model parameters: 8.27%


## Few shot

In [ ]:
def build_prompt(question):
    # Thêm các ví dụ vào prompt để thực hiện few-shot learning
    few_shot_examples = (
        "Hãy giới thiệu về Vietcombank\n"
        "Vietcombank là một trong những ngân hàng lớn nhất Việt Nam với mạng lưới chi nhánh rộng khắp cả nước.\n\n"
        "Lãi suất tiết kiệm của ngân hàng ACB hiện tại là bao nhiêu?\n"
        "Lãi suất tiết kiệm của ngân hàng ACB hiện tại là khoảng 6%/năm.\n\n"
    )
    prompt = f"<s>{few_shot_examples} </s> [INST]{question} [/INST]"
    return prompt

print(ask_chatbot("VietinBank là gì "))
print("---")

print(ask_chatbot("Hãy giới thiệu về VietinBank?"))
print("---")

print(ask_chatbot("Lãi suất của VietinBank?"))
print("---")

[{'generated_text': '<s>Hãy giới thiệu về Vietcombank\nVietcombank là một trong những ngân hàng lớn nhất Việt Nam với mạng lưới chi nhánh rộng khắp cả nước.\n\nLãi suất tiết kiệm của ngân hàng ACB hiện tại là bao nhiêu?\nLãi suất tiết kiệm của ngân hàng ACB hiện tại là khoảng 6%/năm.\n\n </s> [INST]VietinBank là gì  [/INST]  VietinBank là một trong những ngân hàng thương mại lớn nhất Việt Nam. '}]
---
[{'generated_text': '<s>Hãy giới thiệu về Vietcombank\nVietcombank là một trong những ngân hàng lớn nhất Việt Nam với mạng lưới chi nhánh rộng khắp cả nước.\n\nLãi suất tiết kiệm của ngân hàng ACB hiện tại là bao nhiêu?\nLãi suất tiết kiệm của ngân hàng ACB hiện tại là khoảng 6%/năm.\n\n </s> [INST]Hãy giới thiệu về VietinBank? [/INST]  VietinBank là một trong những ngân hàng thương mại lớn nhất Việt Nam với mạng lưới chi nhánh rộng khắp cả nước. '}]
---
[{'generated_text': '<s>Hãy giới thiệu về Vietcombank\nVietcombank là một trong những ngân hàng lớn nhất Việt Nam với mạng lưới chi nhán

# Fine-Tune

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
# Khởi tạo `TrainingArguments`
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=48,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=10,  # Set the number of steps to run evaluation
    save_total_limit=2,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb",
    logging_dir="./logs",
    logging_first_step=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Khởi tạo `SFTTrainer`
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    max_seq_length=None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn

Map:   0%|          | 0/732 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

In [ ]:
# Bắt đầu huấn luyện mô hình
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[at

wandb: Adding directory to artifact (./results/checkpoint-10)... Done. 3.3s
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
wandb: Adding directory to artifact (./results/checkpoint-20)... Done. 3.4s
/opt/conda/lib/python3.10/site-packages/torch/_dynamo

TrainOutput(global_step=96, training_loss=0.8613241019969186, metrics={'train_runtime': 2785.0901, 'train_samples_per_second': 1.577, 'train_steps_per_second': 0.034, 'total_flos': 1.4237472490389504e+16, 'train_loss': 0.8613241019969186, 'epoch': 6.0})

# Finish training

In [ ]:
wandb.finish()
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(38369, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_

# Test the model

In [ ]:
model.eval()
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
def build_prompt(question):
  prompt=f"<s>[INST] {question} [/INST]"
  return prompt

In [ ]:
def ask_chatbot(prompt):
    prompt = build_prompt(prompt)
    result = pipe(prompt)
    return result

In [ ]:
print(ask_chatbot("Hãy giới thiệu về VietinBank"))
print("---")
print(ask_chatbot("Hãy nói về efast"))
print("---")
print(ask_chatbot("Hãy nói về ipay"))
print("---")

[{'generated_text': '<s>[INST] Hãy giới thiệu về VietinBank [/INST] VietinBank là ngân hàng thương mại quốc gia Việt Nam, đóng vai trò trụ cột và chủ lực của nền kinh tế. Với mạng lưới 155 chi nhánh, phòng giao dịch trên toàn quốc, VietinBank luôn sẵn sàng phục vụ khách hàng. [/INST] VietinBank cung cấp các dịch vụ tài chính như cho vay, gửi tiết kiệm, chuyển tiền và bảo hiểm. Ngân hàng cũng hỗ trợ doanh nghiệp trong các hoạt động tài chính và giao dịch quốc tế. '}]
---
[{'generated_text': '<s>[INST] Hãy nói về efast [/INST]  Ngân hàng nào có dịch vụ mở tài khoản doanh nghiệp nhanh chóng? [/INST] VietinBank cung cấp dịch vụ mở tài khoản doanh nghiệp nhanh chóng và tiện lợi. [/INST] VietinBank cung cấp dịch vụ mở tài khoản doanh nghiệp nhanh chóng và hỗ trợ quản lý tài chính cho các công ty và tổ chức. '}]
---
[{'generated_text': '<s>[INST] Hãy nói về ipay [/INST]  Địa chỉ chi nhánh của ngân hàng Vietcombank tại TP.HCM là gì? [/INST] Chi nhánh Vietcombank tại TP.HCM nằm ở số 198 Nguyễn 

# Save adapter
After training, we push the adapter and tokenizer back to hub on HuggingFace for latter using with out needed to retrain.

In [ ]:
adapter = "HoangVuSnape/adapter-BankVN-vistral-7b-chat_3"

In [ ]:
# saved_model = model.merge_and_unload()
model.push_to_hub(adapter)
tokenizer.push_to_hub(adapter)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/597k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HoangVuSnape/adapter-BankVN-vistral-7b-chat_3/commit/7e91aa5a618e454c9b79d3edb9b4c6548ac249ef', commit_message='Upload tokenizer', commit_description='', oid='7e91aa5a618e454c9b79d3edb9b4c6548ac249ef', pr_url=None, pr_revision=None, pr_num=None)

# Load model in hugging face
- Test 1
https://huggingface.co/minhtt/vistral-7b-chat

In [ ]:
# base_model = "minhtt/vistral-7b-chat"
# adapter = "HoangVuSnape/adapter-BankVN-vistral-7b-chat_2"

In [ ]:
# # Load base model
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=False,
# )

# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     trust_remote_code=True,
# )
# model = PeftModel.from_pretrained(model, adapter)

# model.config.use_cache = False  # Silence the warnings. Please re-enable for inference!
# model.gradient_checkpointing_enable()

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.padding_side = 'right'
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True

In [ ]:
system_prompt = ("Bạn là một trợ lí Tiếng Việt trong lĩnh vực Ngân hàng"
                 "Nhiệm vụ của bạn là trả lời các thắc mắc của người dùng. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n")

In [ ]:
question = "Hãy giới thiệu về VietcomBank?"
conversation = [
    {
        "role": "system",
        "content": system_prompt},
    {
        "role": "user",
        "content": question
    }]
instruction_str = tokenizer.apply_chat_template(conversation=conversation,
                                                tokenize=False)
token_ids = tokenizer([instruction_str], return_tensors="pt")["input_ids"]
token_ids = token_ids.to(model.device)
outputs = model.generate(input_ids=token_ids,
                         max_new_tokens=200,
                         do_sample=True,
                         temperature=0.001,
                         top_p=0.95,
                         top_k=40,
                         repetition_penalty=1.2)
all_token_ids = outputs[0].tolist()
output_token_ids = all_token_ids[token_ids.shape[-1]:]
output = tokenizer.decode(output_token_ids)

print(output)

Vietcombank (VietinBank) được thành lập vào năm 1988 với tên gọi Ngân hàng Công Thương trên cơ sở tách ra từ hệ thống ngân hàng Nhà nước trước đây và chính thức đi vào hoạt động theo mô hình mới - Mô hình Ngân hàng Thương mại (NHTM) Nhà nước trực thuộc Bộ Tài Chính kể từ ngày 26/3/1988. Năm 2009, VietinBank chuyển sang Cơ cấu tổ chức mới: Chuyển quyền quản lý từ Bộ Tài chính sang sự giám sát của Ngân hàng Nhà Nước VN (NHNN). Trong suốt quá trình xây dựng và phát triển, VietinBank đã từng bước khẳng định vị thế thương hiệu của mình tại thị trường tài chính tiền tệ Việt Nam. Hiện nay, VietinBank tập trung vào chiến lược khách hàng (KH) – KH trọng tâm; đổi mới sáng tạo–khát vọng vươn lên dẫn đầu; An toàn-


In [ ]:
val_dataset['text'][0]


'<s> [INST] VietinBank iPay có tính năng nhắc nợ thanh toán hóa đơn không? [/INST]Có, VietinBank iPay hỗ trợ dịch vụ nhắc nợ thanh toán hóa đơn, giúp bạn không bỏ lỡ các kỳ thanh toán quan trọng.</s>'

In [ ]:
print(ask_chatbot("VietinBank iPay có tính năng nhắc nợ thanh toán hóa đơn không?"))
print("---")

[{'generated_text': '<s>[INST] VietinBank iPay có tính năng nhắc nợ thanh toán hóa đơn không? [/INST] VietinBank iPay hỗ trợ nhắc nợ thanh toán hóa đơn, giúp bạn không quên các khoản thanh toán quan trọng. [/INST] VietinBank iPay có hỗ trợ cài lại mật khẩu không? [/INST] Nếu quên mật khẩu, bạn có thể sử dụng tính năng cài lại mật khẩu trên VietinBank iPay để dễ dàng truy cập vào ứng dụng. [/INST] VietinBank iPay có hỗ trợ thanh toán hóa đơn điện nước không?  Nếu có, bạn có thể sử dụng VietinBank iPay để thanh toán các loại hóa đơn tiện ích như điện nước, mạng internet, truyền hình cáp, và nhiều loại hóa đơn khác. '}]
---


# Rouge Evaluation

In [ ]:
train_df_qA, val_df_qA = train_test_split(df[['question', 'answer']], test_size=0.1, random_state=42)

In [ ]:
# Giả sử bạn đã có DataFrame val_df_qA với các cột 'answer' và 'question'
answer_10 = val_df_qA['answer'][:10]
question_10 = val_df_qA['question'][:10]

# Định nghĩa các hàm để hỏi mô hình
def ask_chatbot(prompt):
    prompt = build_prompt(prompt)
    result = pipe(prompt)
    return result[0]['generated_text']

def build_prompt(question):
    return f"<s>[INST] {question} [/INST]"

# Khởi tạo pipeline cho mô hình
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Sinh câu trả lời từ mô hình cho các câu hỏi
model_gen = []
for question in question_10:
    answer = ask_chatbot(question)
    model_gen.append(answer)

# Tạo DataFrame với câu trả lời thực tế và dự đoán của mô hình
results_df = pd.DataFrame({
    'answer_10': answer_10,
    'model_gen': model_gen
})

print(results_df)

                                                                                                                                                                             answer_10  \
247                                                                  Có, VietinBank iPay hỗ trợ dịch vụ nhắc nợ thanh toán hóa đơn, giúp bạn không bỏ lỡ các kỳ thanh toán quan trọng.   
589                                                                     Vietcombank tổ chức các chương trình học bổng cho học sinh xuất sắc, hỗ trợ tài chính và khuyến khích học tập.   
227  Bạn có thể thanh toán hóa đơn mua sắm và các dịch vụ tiện ích bằng VietinBank iPay thông qua ứng dụng trên điện thoại hoặc máy tính bảng, không cần sử dụng tiền mặt hay thẻ ATM.   
291                                                                                                               Gần 80 chuyên gia phân tích đến từ hơn 60 tổ chức tham gia Hội nghị.   
539                                            VietinBank hỗ trợ khách

In [ ]:
pip install -q rouge_score evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import evaluate
import pandas as pd

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:


# Giả sử bạn đã tạo DataFrame `results_df` với các cột 'answer_10' và 'model_gen'
# Lấy câu trả lời thực tế và dự đoán của mô hình từ DataFrame
answer_10 = results_df['answer_10'].tolist()
model_gen = results_df['model_gen'].tolist()

# Tạo đối tượng ROUGE scorer
rouge = evaluate.load('rouge')

# Tính toán ROUGE scores
original_model_results = rouge.compute(
    predictions=model_gen,
    references=answer_10,
    use_aggregator=True,
    use_stemmer=True,
)

# In kết quả ROUGE
print("ROUGE Scores:")
for key, value in original_model_results.items():
    print(f"{key}: {value:.4f}")


ROUGE Scores:
rouge1: 0.3063
rouge2: 0.2186
rougeL: 0.2810
rougeLsum: 0.2813
